## H1B Sponsorsip: Big-Name Compaines v.s. Start-ups

As an international student who just started the Master of Science in Business Analytics, I have an ultimate goal of finding a data analyst job within the US. Past experience told me that I have liked the atmosphere and style of start-ups better. However, I have also heard a lot that with limited resource, start-ups tend not to hire international students who need H1B sponsorship. Thus began my research: What is the H1B application like when taking company size into consideration? Do titles give a huge difference in salary? What are my best chance to land a job in the US?

In order to answer my question, I utilized the technique of web scraping on [H1B Salary Database](https://h1bdata.info/index.php).

In [1]:
import pandas as pd
import numpy as np
import re
import us
from urllib.request import urlopen
from bs4 import BeautifulSoup

### Retrieving Data Needed

Since different companies with different job titles might be doing similar work, I have chosen a few common job titles for MSBA students, including:

* Analyst
* Data Analyst
* Business Analyst
* Business Intelligence Engineer
* Business Intelligence Developer
* Business Intelligence Analyst
* Data Scientist
* Data Scientist Analyst
* Data Specialist
* Business Data Analyst

Since there are unlimited job titles that are possible, above mentioned are just a few more common ones.

In [2]:
#create the list of desired job title search url
urls = ['https://h1bdata.info/index.php?em=&job=analyst&city=&year=All+Years',
        'https://h1bdata.info/index.php?em=&job=data+analyst&city=&year=All+Years', 
        'https://h1bdata.info/index.php?em=&job=business+analyst&city=&year=All+Years', 
        'https://h1bdata.info/index.php?em=&job=Business+Intelligence+Engineer&city=&year=All+Years', 
        'https://h1bdata.info/index.php?em=&job=Business+Intelligence+Developer&city=&year=All+Years',
        'https://h1bdata.info/index.php?em=&job=Business+Intelligence+Analyst&city=&year=All+Years',
        'https://h1bdata.info/index.php?em=&job=Data+Scientist+I&city=&year=All+Years', 
        'https://h1bdata.info/index.php?em=&job=Data+Scientist+1&city=&year=All+Years', 
        'https://h1bdata.info/index.php?em=&job=Data+Scientist+Analyst&city=&year=All+Years', 
        'https://h1bdata.info/index.php?em=&job=Data+Specialist&city=&year=All+Years', 
        'https://h1bdata.info/index.php?em=&job=Business+Data+Analyst&city=&year=All+Years']

#url = 'https://h1bdata.info/index.php?em=&job=data+analyst&city=&year=All+Years'

In [3]:
#using for loop to retrieve data 

record = []

for url in urls:
    print('Retrieving:', url)
    html = urlopen(url).read()
    soup = BeautifulSoup(html, "lxml")
    data = soup.tbody.find_all("tr")

    #finding all "td" in html since "td" tag represents data I need
    for index in range(len(data)):
        for td in data[index].find_all("td"):
            try:
                record.append(td.text.replace('\n', ' ').strip())
            except:
                continue
    print('Total Records:', int(len(record)/7))

Retrieving: https://h1bdata.info/index.php?em=&job=analyst&city=&year=All+Years
Total Records: 19106
Retrieving: https://h1bdata.info/index.php?em=&job=data+analyst&city=&year=All+Years
Total Records: 27168
Retrieving: https://h1bdata.info/index.php?em=&job=business+analyst&city=&year=All+Years
Total Records: 61903
Retrieving: https://h1bdata.info/index.php?em=&job=Business+Intelligence+Engineer&city=&year=All+Years
Total Records: 62222
Retrieving: https://h1bdata.info/index.php?em=&job=Business+Intelligence+Developer&city=&year=All+Years
Total Records: 63537
Retrieving: https://h1bdata.info/index.php?em=&job=Business+Intelligence+Analyst&city=&year=All+Years
Total Records: 67897
Retrieving: https://h1bdata.info/index.php?em=&job=Data+Scientist+I&city=&year=All+Years
Total Records: 68286
Retrieving: https://h1bdata.info/index.php?em=&job=Data+Scientist+1&city=&year=All+Years
Total Records: 68407
Retrieving: https://h1bdata.info/index.php?em=&job=Data+Scientist+Analyst&city=&year=All+Ye

In [4]:
#cleaning scraped data by reshaping and deleting unwanted rows
data = np.reshape(record, (int(len(record)/7) ,7))
df = pd.DataFrame(data, columns = ['name', 'title', 'base_salary', 'location', 'submit_date', 'start_date', 'status'])

df['name'] = df['name'].str.title()
df['title'] = df['title'].str.title()
df['status'] = df['status'].str.title()

df['base_salary'] = df['base_salary'].str.replace(',', '')
df['base_salary'] = df['base_salary'].astype(int)

df['submit_date'] = pd.to_datetime(df['submit_date'])
df['start_date'] = pd.to_datetime(df['start_date'])

df['location'] = df['location'].str.split(',')
df['city'] = df['location'].str[0]
df['city'] = df['city'].str.title()
df['state'] = df['location'].str[1]
df['state'] = df['state'].str.strip()
df['city'] = df['city'].str.strip()
#mapping US states to full name using us package
df['state'] = df['state'].map(us.states.mapping('abbr', 'name')).fillna(df['state']).str.title()

#clean error data in state column and remove unwanted
df.loc[df['state'] == 'San Ramon', 'state'] = 'California'
df.loc[df['state'] == 'Ca 95134', 'state'] = 'California'
df.loc[df['state'] == 'Ca 90802', 'state'] = 'California'
df.loc[df['state'] == 'Woodland Hills', 'state'] = 'California'
df.loc[df['state'] == 'Dc 20006', 'state'] = 'District Of Columbia'
df.loc[df['state'] == 'D.C.', 'state'] = 'District Of Columbia'
df.loc[df['state'] == 'Ets Drive', 'state'] = 'New Jersey'
df.loc[df['state'] == 'Mo 63105', 'state'] = 'Missouri'
df.loc[df['state'] == 'Minneapolis', 'state'] = 'Minnesota'
df.loc[df['state'] == 'Long Island City', 'state'] = 'New York'
df.loc[df['state'] == 'Atlanta', 'state'] = 'Georgia'
df = df[df['state'] != '']
df = df[df['state'] != 'Puerto Rico']
df = df[df['state'] != 'Guam']
df = df[df['state'] != 'Virgin Islands']
df['state'] = df['state'].astype('category')

df['name'] = df['name'].str.replace('Ltd', '')
df['name'] = df['name'].str.replace('Llc', '')
df['name'] = df['name'].str.replace('Inc', '')
df['name'] = df['name'].str.strip()

#remove location column since no longer needed
df.drop(columns = ['location'], inplace = True)
df['category'] = None

#select only the ones that are submitted after 2018-01-01
df = df[df['submit_date'] >= '2018-01-01'].reset_index(drop = True)

#consider ones that have more than 20 applications since 2018
count = df.groupby('name').count().sort_values('title', ascending = False)
select = count[count['title'] >= 20].index.tolist()
result = df[df['name'].isin(select)]

In [5]:
result = result.reset_index()
result.drop(labels='index', axis='columns', inplace=True)
result

,name,title,base_salary,submit_date,start_date,status,city,state,category
0,Rizontek,Analyst,54000,2020-05-06,2020-10-01,Certified,Austin,Texas,None
1,Rizontek,Analyst,54000,2020-05-14,2020-10-01,Certified,Austin,Texas,None
2,Rizontek,Analyst,54000,2019-02-15,2019-08-17,Certified,Syracuse,New York,None
3,Rizontek,Analyst,56000,2019-02-14,2019-08-16,Certified,Austin,Texas,None
4,Turnberry Solutions,Analyst,56020,2019-11-05,2020-01-01,Certified,Malvern,Pennsylvania,None
...,...,...,...,...,...,...,...,...,...
17691,Tribolatech,Business Data Analyst,120000,2020-03-23,2020-04-13,Certified,Tempe,Arizona,None
17692,Tribolatech,Business Data Analyst,120000,2020-03-25,2020-04-13,Certified,Tempe,Arizona,None
17693,Intuit,Business Data Analyst,120967,2019-05-09,2019-05-15,Certified,Mountain View,California,None
17694,Exzac,Business Data Analyst,125000,2020-09-09,2020-10-01,Certified,New York,New York,None


In [ ]:
for tuples in matches:
    if tuples[1] >= 80:
        print(tuples[0], matches[0])

In [ ]:
matches[0]

### Is This Company A Start-up or A Big-Name?

In order to decide whether or not the company is a "big-name" company, I decided to utilize the information on Wikipedia regarding [unicorn start-up companies](https://en.wikipedia.org/wiki/List_of_unicorn_startup_companies) and [2019 Fortune 500 companies](https://docs.google.com/spreadsheets/d/1SlNYZxOvpuQejYWhM7S81v3Hi_BqfbynVqwiS_XUTBI/edit#gid=0) (parsed by [Ashely Ng on towards data science](https://towardsdatascience.com/scraping-the-fortune-500-company-job-boards-step-by-step-a124cf8bc364))

In [ ]:
list = pd.Series(df['name'].unique()).tolist()
list.sort()
list